****

**Decision Tree**

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder 

**Lets build a simple dataset**

X_train contains three features for each example:<br>
Brown Color (A value of 1 indicates "Brown" cap color and 0 indicates "Red" cap color)<br>
Tapering Shape (A value of 1 indicates "Tapering Stalk Shape" and 0 indicates "Enlarging" stalk shape)<br>
Solitary (A value of 1 indicates "Yes" and 0 indicates "No")<br><br>
y_train is whether the mushroom is edible<br>
y = 1 indicates edible<br>
y = 0 indicates poisonous

|                                                    | Brown Cap | Tapering Stalk Shape | Solitary | Edible |
|:--------------------------------------------------:|:---------:|:--------------------:|:--------:|:------:|
| <img src="images/0.png" alt="drawing" width="50"/> |     1     |           1          |     1    |    1   |
| <img src="images/1.png" alt="drawing" width="50"/> |     1     |           0          |     1    |    1   |
| <img src="images/2.png" alt="drawing" width="50"/> |     1     |           0          |     0    |    0   |
| <img src="images/3.png" alt="drawing" width="50"/> |     1     |           0          |     0    |    0   |
| <img src="images/4.png" alt="drawing" width="50"/> |     1     |           1          |     1    |    1   |
| <img src="images/5.png" alt="drawing" width="50"/> |     0     |           1          |     1    |    0   |
| <img src="images/6.png" alt="drawing" width="50"/> |     0     |           0          |     0    |    0   |
| <img src="images/7.png" alt="drawing" width="50"/> |     1     |           0          |     1    |    1   |
| <img src="images/8.png" alt="drawing" width="50"/> |     0     |           1          |     0    |    1   |
| <img src="images/9.png" alt="drawing" width="50"/> |     1     |           0          |     0    |    0   |

In [39]:
X_train = np.array([[1,1,1],[1,0,1],[1,0,0],[1,0,0],[1,1,1],[0,1,1],[0,0,0],[1,0,1],[0,1,0],[1,0,0]])
y_train = np.array([1,1,0,0,1,0,0,1,1,0])
root_indices=np.array([0,1,2,3,4,5,6,7,8,9])

In [31]:
X_train.shape

(10, 3)

We have three feature and ten data samples

Steps for building a decision tree are as follows:<br>
-Start with all examples at the root node,<br>
-Calculate information gain for splitting on all possible features, and pick the one with the highest information gain,<br>
-Split dataset according to the selected feature, and create left and right branches of the tree,<br>
-Keep repeating splitting process until stopping criteria is met.<br>
<br>
In this lab, you'll implement the following functions, which will let you split a node into left and right branches using the feature with the
 highest information gain<br>
-Calculate the entropy at a node<br>
-Split the dataset at a node into left and right branches based on a given feature<br>
-Calculate the information gain from splitting on a given feature<br>
-Choose the feature that maximizes information gain<br>
<br><br>
For this lab, the stopping criteria we've chosen is setting a maximum depth of 2

Entropy is the minimum number of bits needed in order to classify an arbitrary
example as yes or no. It is calculated as:

First, we'll write a helper function called `compute_entropy` that computes the entropy (measure of impurity) at a node. 
- The function takes in a numpy array (`y`) that indicates whether the examples in that node are edible (`1`) or poisonous(`0`) 
- `Entropy` is the minimum number of bits needed in order to classify an arbitrary example as yes or no
- Entropy(S)= expected number of bits needed to encode class (+ or -) of randomly drawn members of S. It depends from the distribution of the
  random variable p.
- `S` is a collection of training examples
- `p_1` the proportion of positive examples in S
- `1-p_1` the proportion of negative examples in S
 

$$H(p_1) = -p_1 \text{log}_2(p_1) - (1- p_1) \text{log}_2(1- p_1)$$
* Note 
    * The log is calculated with base $2$
    * For implementation purposes, $0\text{log}_2(0) = 0$. That is, if `p_1 = 0` or `p_1 = 1`, set the entropy to `0`
    * Make sure to check that the data at a node is not empty (i.e. `len(y) != 0`). Return `0` if it is

#Entropy is best explained in this video:
https://www.youtube.com/watch?v=YtebGVx-Fxw

In [50]:
def compute_entropy(y):
    #y is ndarray of the target value 
    entropy = 0.
    ones=0
    zeros=0
    len_y=len(y)
    if len_y!=0:
        ones=np.count_nonzero(y==1)
        p1=ones/len_y
        if p1 != 1:
            if p1 !=0:
                entropy=-p1*np.log2(p1)-(1-p1)*np.log2(1-p1)      
    return entropy

###  Split dataset

Next, we'll write a helper function called `split_dataset` that takes in the data at a node and a feature to split on and splits it into left 
and right branches.

- The function takes in the training data, the list of indices of data points at that node, along with the feature to split on. 
- It splits the data and returns the subset of indices at the left and the right branch.
- For example, say we're starting at the root node (so `node_indices = [0,1,2,3,4,5,6,7,8,9]`), and we chose to split on feature `0`, which is  
whether or not the example has a brown cap. 
    - The output of the function is then, `left_indices = [0,1,2,3,4,7,9]` (data points with brown cap) and `right_indices = [5,6,8]` (data 
    points without a brown cap)

|                                                    | Brown Cap | Tapering Stalk Shape | Solitary | Edible |
|:--------------------------------------------------:|:---------:|:--------------------:|:--------:|:------:|
| <img src="images/0.png" alt="drawing" width="50"/> |     1     |           1          |     1    |    1   |
| <img src="images/1.png" alt="drawing" width="50"/> |     1     |           0          |     1    |    1   |
| <img src="images/2.png" alt="drawing" width="50"/> |     1     |           0          |     0    |    0   |
| <img src="images/3.png" alt="drawing" width="50"/> |     1     |           0          |     0    |    0   |
| <img src="images/4.png" alt="drawing" width="50"/> |     1     |           1          |     1    |    1   |
| <img src="images/5.png" alt="drawing" width="50"/> |     0     |           1          |     1    |    0   |
| <img src="images/6.png" alt="drawing" width="50"/> |     0     |           0          |     0    |    0   |
| <img src="images/7.png" alt="drawing" width="50"/> |     1     |           0          |     1    |    1   |
| <img src="images/8.png" alt="drawing" width="50"/> |     0     |           1          |     0    |    1   |
| <img src="images/9.png" alt="drawing" width="50"/> |     1     |           0          |     0    |    0   |

In [49]:
def split_dataset(X, node_indices, feature):
    """
    Args:
        X (ndarray):             Data matrix of shape(n_samples, n_features)
        node_indices (list):     List containing the active indices. I.e, the samples being considered at this step.
        feature (int):           Index of feature to split on
    
    Returns:
        left_indices (list):     Indices with feature value == 1
        right_indices (list):    Indices with feature value == 0
    """
    left_indices = []
    right_indices = []
    for i in range(len(node_indices)):
        if X[node_indices[i]][feature]==1:
            left_indices.append(node_indices[i])  
        else:
            right_indices.append(node_indices[i])       
    return left_indices, right_indices

## Information gain
`Information gain` is the expected reduction in entropy caused by `partitioning` the examples on an attribute.<br>
The `higher` the information gain the more `effective` the attribute in `classifying` training data.

In [54]:
def compute_information_gain(X, y, node_indices, feature):
    
    """
    Compute the information of splitting the node on a given feature
    
    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.
   
    Returns:
        cost (float):        Cost computed
    
    """    
    # Split dataset
    left_indices, right_indices = split_dataset(X, node_indices, feature)
    
    # Some useful variables
    X_node, y_node = X[node_indices], y[node_indices]
    X_left, y_left = X[left_indices], y[left_indices]
    X_right, y_right = X[right_indices], y[right_indices]
    information_gain = 0

    h_p_node=compute_entropy(y_node)
    h_p_left=compute_entropy(y_left)
    h_p_right=compute_entropy(y_right)
    w_left=len(y_left)/len(y_node)
    w_right=len(y_right)/len(y_node)
    
    information_gain=h_p_node - (w_left*h_p_left + w_right*h_p_right)

    return information_gain


**To get best split we have to choose the feature with `high information gain.`**

In [52]:
def get_best_split(X, y, node_indices):   
    """
    Returns the optimal feature and threshold value
    to split the node data 
    
    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.

    Returns:
        best_feature (int):     The index of the best feature to split
    """    
    
    # Some useful variables
    num_features = X.shape[1]
    en_temp=0.
    # You need to return the following variables correctly
    best_feature = -1
    best_feature_indices=-1
    for i in range(num_features):
        en_temp=compute_information_gain(X, y, node_indices, feature=i)
        if en_temp!=0 and en_temp > best_feature:
            best_feature_indices=i 
    return best_feature_indices

In [40]:
tree=[]
def build_tree_recursive(X, y, node_indices, branch_name, max_depth, current_depth):
    """
    Build a tree using the recursive algorithm that split the dataset into 2 subgroups at each node.
    This function just prints the tree.
    
    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.
        branch_name (string):   Name of the branch. ['Root', 'Left', 'Right']
        max_depth (int):        Max depth of the resulting tree. 
        current_depth (int):    Current depth. Parameter used during recursive call.
   
    """ 

    # Maximum depth reached - stop splitting
    if current_depth == max_depth:
        formatting = " "*current_depth + "-"*current_depth
        print(formatting, "%s leaf node with indices" % branch_name, node_indices)
        return
   
    # Otherwise, get best split and split the data
    # Get the best feature and threshold at this node
    best_feature = get_best_split(X, y, node_indices) 
    
    formatting = "-"*current_depth
    print("%s Depth %d, %s: Split on feature: %d" % (formatting, current_depth, branch_name, best_feature))
    
    # Split the dataset at the best feature
    left_indices, right_indices = split_dataset(X, node_indices, best_feature)
    tree.append((left_indices, right_indices, best_feature))
    
    # continue splitting the left and the right child. Increment current depth
    build_tree_recursive(X, y, left_indices, "Left", max_depth, current_depth+1)
    build_tree_recursive(X, y, right_indices, "Right", max_depth, current_depth+1)

In [55]:
build_tree_recursive(X_train, y_train, root_indices, "Root", max_depth=2, current_depth=0)

 Depth 0, Root: Split on feature: 2
- Depth 1, Left: Split on feature: 1
  -- Left leaf node with indices [0, 4, 5]
  -- Right leaf node with indices [1, 7]
- Depth 1, Right: Split on feature: 1
  -- Left leaf node with indices [8]
  -- Right leaf node with indices [2, 3, 6, 9]


In [56]:
tree

[([0, 1, 4, 5, 7], [2, 3, 6, 8, 9], 2),
 ([0, 4, 5], [1, 7], 1),
 ([8], [2, 3, 6, 9], 1)]